# Trip Distribtuion
Example of doubly constrained gravity model for 21-zone region

In [75]:
import pandas as pd
import numpy as np

In [76]:
# Load production and attraction totals by zone
prodattr = pd.read_csv(r'T:\CEE581\Assignments\Labs\Lab 2 - Trip Distribution\Class Example in Python\ProdAttr.csv',
                      index_col='Zone')

# Load friction factors (calcualted in Excel sheet)
df_ff = pd.read_csv(r'T:\CEE581\Assignments\Labs\Lab 2 - Trip Distribution\Class Example in Python\friction_matrix.csv',
                index_col=0)
df_ff.columns = df.columns.astype('int')

In [79]:
# HBW target is productions, want to balance attractions to productions
# Should already be balanced totals

print prodattr['HBW_P'].sum()
print prodattr['HBW_A'].sum()

1962
1962


In [106]:
# Create an array to hold a and b values
# These will be updated iteratively
# They both start at values of 1
a_b_array = pd.DataFrame(1, index=xrange(1,tot_zones+1), columns=['a','b'])

## Set b(j)=1 and calculate a(i)

In [108]:
# Calculate the matrix b(j)D(j)f(ij)
# This is used to calculate A(i) as 1/b*D*f

# Create an empty matrix
bdf = pd.DataFrame(np.nan, index=xrange(1,tot_zones+1), columns=xrange(1,tot_zones+1))

# Loop through origins (i)
for i in xrange(1,tot_zones+1):
    # Loop through destinations (j)
    for j in xrange(1,tot_zones+1):
        
        dest_tot = prodattr['HBW_A'].loc[j]    # Select the j column (destination)     
        ff = df_ff.ix[i,j] # friction factor value for ij
        b_value = a_b_array['b'].loc[j]    
        bdf.ix[i,j] = b_value*dest_tot*ff
        
# Calcualte a as 1/bdf
a_b_array['a'] = 1/bdf.sum(axis=1)

### Calculate total trips and see how close we are to targets

In [109]:
trip_table = pd.DataFrame(np.nan, index=xrange(1,tot_zones+1), columns=xrange(1,tot_zones+1))

# Loop through origins (i)
for i in xrange(1,tot_zones+1):
    # Loop through destinations (j)
    for j in xrange(1,tot_zones+1):
        # a_value
        a_value = a_b_array['a'].loc[i]
        origin_totals = prodattr['HBW_P'].loc[i]
        b_value = a_b_array['b'].loc[j]
        dest_totals = prodattr['HBW_A'].loc[j]
        ff = df_ff.ix[i,j]
        
        trip_table.ix[i,j] = a_value*origin_totals*b_value*dest_totals*ff

In [189]:
# Calculate percent difference for origins
(trip_table.sum(axis=1)-prodattr['HBW_P'])/prodattr['HBW_P']

1   -0.007650
2   -0.023913
3    0.006112
4    0.016546
dtype: float64

In [188]:
(trip_table.sum(axis=0)-prodattr['HBW_A'])/prodattr['HBW_A']

1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

In [ ]:
# What is our stopping criteria?

## Fix a(i) and solve for b(j)

In [133]:
# Calculate the matrix a(i)O(i)f(ij)
# This is used to calculate B(j) as 1/a*O*f
adf = pd.DataFrame(np.nan, index=xrange(1,tot_zones+1), columns=xrange(1,tot_zones+1))

# Loop through origins (i)
for i in xrange(1,tot_zones+1):
    # Loop through destinations (j)
    for j in xrange(1,tot_zones+1):
        origin_tot = prodattr['HBW_P'].loc[i]    # Select the i column (origins)     
        ff = df_ff.ix[i,j] # friction factor value for ij
        a_value = a_b_array['a'].loc[i]    
        adf.ix[i,j] = a_value*origin_tot*ff

In [135]:
adf

,1,2,3,4
1,0.595903,0.267756,0.132964,0.089128
2,0.214742,0.528181,0.214742,0.106638
3,0.098067,0.125921,0.280242,0.229443
4,0.079525,0.144903,0.393888,0.531693


# Create functions to replicate and iterate
Generalize process above into functions
Find what is common in each process
Differences should be passed in as variables

In [136]:
# prodattr['HBW_A'].loc[1fe]

Zone
1    260
2    400
3    500
4    802
Name: HBW_A, dtype: int64

In [177]:
def update_a(a_b_array, prodattr, df_ff):
    """
    Calculate the matrix b(j)D(j)f(ij)
    This is used to calculate A(i) as 1/b*D*f
    """

    # Create an empty matrix
    bdf = pd.DataFrame(np.nan, index=xrange(1,tot_zones+1), columns=xrange(1,tot_zones+1))

    # Loop through origins (i)
    for i in xrange(1,tot_zones+1):
        # Loop through destinations (j)
        for j in xrange(1,tot_zones+1):

            dest_tot = prodattr['HBW_A'].loc[j]    # Select the j column (destination)     
            ff = df_ff.ix[i,j] # friction factor value for ij
            b_value = a_b_array['b'].loc[j]    
            bdf.ix[i,j] = b_value*dest_tot*ff
        
    # Calcualte a as 1/bdf
    a = 1/bdf.sum(axis=1)
    
    return a
    

In [178]:
def update_b(a_b_array, prodattr, df_ff):
    """
    Calculate the matrix a(i)O(i)f(ij)
    This is used to calculate B(j) as 1/a*O*f
    """
    adf = pd.DataFrame(np.nan, index=xrange(1,tot_zones+1), columns=xrange(1,tot_zones+1))

    # Loop through origins (i)
    for i in xrange(1,tot_zones+1):
        # Loop through destinations (j)
        for j in xrange(1,tot_zones+1):
            origin_tot = prodattr['HBW_P'].loc[i]    # Select the i column (origins)     
            ff = df_ff.ix[i,j] # friction factor value for ij
            a_value = a_b_array['a'].loc[i]    
            adf.ix[i,j] = a_value*origin_tot*ff
            
    # Calcualte b as 1/adf
    b = 1/adf.sum(axis=0)
    
    return b

With these two functions we can easily update a and b

In [180]:
a_b_array = pd.DataFrame(1, index=xrange(1,tot_zones+1), columns=['a','b'])

a = update_a(a_b_array=a_b_array,
        prodattr=prodattr,
        df_ff=df_ff)

a_b_array['a'] = a

b = update_b(a_b_array=a_b_array,
        prodattr=prodattr,
        df_ff=df_ff)

a_b_array['b'] = b

We now need to iterate through the a and b updating until the trip table is balanced.
Let's make a function that calculates the trip table and checks if we're close enough.

In [184]:
def create_trip_table(a_b_array, prodattr, df_ff):
    """
    Calculate trip tables given origin and destination totals, 
    friction factors, and arrays a & b for origin and destination balancing
    """
    # Fill an initially empty trip table
    trip_table = pd.DataFrame(np.nan, index=xrange(1,tot_zones+1), columns=xrange(1,tot_zones+1))

    # Loop through origins (i)
    for i in xrange(1,tot_zones+1):
        # Loop through destinations (j)
        for j in xrange(1,tot_zones+1):
            # a_value
            a_value = a_b_array['a'].loc[i]
            origin_totals = prodattr['HBW_P'].loc[i]
            b_value = a_b_array['b'].loc[j]
            dest_totals = prodattr['HBW_A'].loc[j]
            ff = df_ff.ix[i,j]

            trip_table.ix[i,j] = a_value*origin_totals*b_value*dest_totals*ff
            
    return trip_table

In [186]:
trip_table = create_trip_table(a_b_array, prodattr, df_ff)
trip_table

,1,2,3,4
1,156.778891,100.399696,65.061215,74.700316
2,56.497587,198.050220,105.076646,89.375426
3,25.800996,47.216140,137.126685,192.300823
4,20.922527,54.333943,192.735455,445.623435


From the lab assignment, the difference between any of your calculated row/column total and the target row/column total should be less than 10

In [191]:
def check_convergence(trip_table, prodattr):
    """
    Calcualte difference between current trip table Ps and As with targets
    """
    df = pd.DataFrame()
    df['production_difference'] = trip_table.sum(axis=1)-prodattr['HBW_P']
    df['attraction_difference'] = trip_table.sum(axis=0)-prodattr['HBW_A']
    
    return df

In [257]:
df_check = check_convergence(trip_table, prodattr)
df_check

,production_difference,attraction_difference
1,-1.725342,5.684342e-14
2,-1.854384,-5.684342e-14
3,0.879788,-1.136868e-13
4,2.699938,0.000000e+00


In [258]:
# If any absolute difference is >= 10, continue iterating
if (abs(df_check.values) >= 10).any() != True:
    print 'continue'
else:
    print 'converged'

continue


Now we should have all the functions we need to write a full progrm

In [267]:
# Run the program until we can show the numbered are converged
# Initialize this variable and change to True when convergence is reached
converged = False
converged_critieria = 0.1 # absolute difference between P and A totals

# Initialzie an array of ones for a_b_array
a_b_array = pd.DataFrame(1, index=xrange(1,tot_zones+1), columns=['a','b'])

# Count iterations
iteration_number = 0

while converged is False:
    print 'calculating iteration number ' + str(iteration_number)
    
    # Calculate a with a fixed b
    a = update_a(a_b_array=a_b_array,
        prodattr=prodattr,
        df_ff=df_ff)
    a_b_array['a'] = a

    # Calculate b with a fixed a
    b = update_b(a_b_array=a_b_array,
            prodattr=prodattr,
            df_ff=df_ff)
    a_b_array['b'] = b
    
    # Compute trip_table and check convergence
    trip_table = create_trip_table(a_b_array, prodattr, df_ff)
    df_check = check_convergence(trip_table, prodattr)
    
    # If any absolute difference is < converged_critieria, program is converged; stop running
    if (abs(df_check.values) >= converged_critieria).any() != True:
        converged = True
    else:
        iteration_number += 1

calculating iteration number 0
calculating iteration number 1
calculating iteration number 2
calculating iteration number 3
calculating iteration number 4


In [268]:
trip_table.sum(axis=0)

1    260.0
2    400.0
3    500.0
4    802.0
dtype: float64

In [269]:
trip_table.sum(axis=1)

1    399.959910
2    459.972020
3    400.017247
4    702.050822
dtype: float64

# Apply program to the lab dataset

In [299]:
# Load provided files
prodattr = pd.read_csv(r'T:\CEE581\Assignments\Labs\Lab 2 - Trip Distribution\ProdAttr.csv',index_col='Zone')
imped = pd.read_csv(r'T:\CEE581\Assignments\Labs\Lab 2 - Trip Distribution\imped.csv')
ff = pd.read_csv(r'T:\CEE581\Assignments\Labs\Lab 2 - Trip Distribution\frictionfactors.csv')

In [342]:
imped['Trv_Time'] = imped['Trv_Time'].astype('int')


# Add the friction factor to the impedance file
# Join based on travel time field
imped = pd.merge(imped,ff,left_on='Trv_Time',right_on='TRV_TIME')

# Pivot as a matrix instead of flat file
df = imped[['FromZone','ToZone','HBW_FF']]
df_ff = pd.pivot_table(df,index='FromZone',columns='ToZone',values='HBW_FF')

In [343]:
# Run the program until we can show the numbered are converged
# Initialize this variable and change to True when convergence is reached
converged = False
converged_critieria = 10 # absolute difference between P and A totals

# Initialzie an array of ones for a_b_array
a_b_array = pd.DataFrame(1, index=xrange(1,tot_zones+1), columns=['a','b'])

# Count iterations
iteration_number = 0

while converged is False:
    print 'calculating iteration number ' + str(iteration_number)
    
    # Calculate a with a fixed b
    a = update_a(a_b_array=a_b_array,
        prodattr=prodattr,
        df_ff=df_ff)
    a_b_array['a'] = a

    # Calculate b with a fixed a
    b = update_b(a_b_array=a_b_array,
            prodattr=prodattr,
            df_ff=df_ff)
    a_b_array['b'] = b
    
    # Compute trip_table and check convergence
    trip_table = create_trip_table(a_b_array, prodattr, df_ff)
    df_check = check_convergence(trip_table, prodattr)
    
    # If any absolute difference is < converged_critieria, program is converged; stop running
    if (abs(df_check.values) >= converged_critieria).any() != True:
        converged = True
    else:
        iteration_number += 1

calculating iteration number 0
calculating iteration number 1
calculating iteration number 2
calculating iteration number 3


In [344]:
trip_table

,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
1,1091.356212,74.461206,52.725708,23.051637,3.322263,30.950634,36.410586,15.119522,2.413387,1.627085,...,5.810675,1.170376,4.991530,2.661969,1.089758,1.412144,4.775728,2.088091,7.571742,5.404418
2,223.590707,403.442732,115.222785,49.612047,10.583085,8.967050,12.329899,5.545353,1.797966,1.523875,...,3.211791,1.121049,35.176656,13.521540,7.691366,6.487598,33.355339,8.768049,36.263090,18.648012
3,263.793822,114.975974,734.814149,234.252196,32.929498,40.438582,38.922780,16.715284,3.657728,3.036957,...,6.857239,2.512185,26.439279,14.228176,13.488862,13.837785,70.196981,22.321616,77.729670,37.163935
4,221.636737,81.383115,205.588526,832.251279,81.894352,122.917843,59.155193,23.430503,4.391474,3.625340,...,9.744960,3.802951,19.726030,10.812055,10.250247,17.207011,63.041731,46.260809,193.310598,101.667751
5,73.146822,34.717716,50.805875,89.565676,213.657006,88.188267,57.874510,20.064000,4.275941,3.793162,...,24.033591,12.667820,14.152579,7.051986,4.947312,12.345290,38.271311,19.914085,113.475307,99.466691
6,671.931871,51.864669,105.803502,182.225357,94.044525,752.823596,135.863387,57.285330,13.207884,11.194407,...,66.678287,25.163226,22.887580,16.554900,8.632078,17.710736,60.440421,32.205100,133.194558,96.514643
7,179.803687,14.194000,23.164515,22.710168,17.844681,26.783650,158.644252,48.774422,7.518475,8.355316,...,20.213279,3.105192,5.980852,3.986965,2.542773,4.371080,19.670329,6.254878,23.977222,14.996067
8,20.657344,2.066587,3.262278,3.127775,1.734738,2.603724,8.892706,28.405404,2.593498,2.497878,...,2.517873,0.561483,0.906089,0.650144,0.350205,0.718528,2.697289,1.019966,3.302281,2.431474
9,288.944976,49.285447,55.838152,40.687325,26.539500,43.248323,120.499793,115.471447,255.585170,46.155358,...,45.904498,15.216640,15.626390,11.212360,4.745425,9.346900,27.658985,14.449181,42.957367,32.006109
10,253.717710,45.322528,57.529166,41.919508,27.272207,44.771261,160.059912,155.740462,53.862006,276.672796,...,90.638451,27.986216,19.068644,14.402390,6.186980,12.256317,32.197474,16.946197,50.293519,38.089219
